In [21]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Load datasets
training_data = pd.read_csv("training_data.csv")
target_data = pd.read_csv("target.csv")

# Concatenate the dataframes for later usage (axis=0)
data = pd.concat([training_data, target_data], axis=0, ignore_index=True)


len_texts = [len(x) for x in data['cleaned']]
data

,filename,narrative,sub_narrative,text,language,narrative_labels,sub_narrative_labels,cleaned
0,EN_CC_100013.txt,CC: Criticism of climate movement,CC: Criticism of climate movement: Ad hominem ...,Bill Gates Says He Is ‘The Solution’ To Climat...,EN,['CC: Criticism of climate movement'],['CC: Criticism of climate movement: Ad homine...,bill gates say solution climate change ok priv...
1,EN_UA_300009.txt,Other,Other,Russia: Clashes erupt in Bashkortostan as righ...,EN,['Other'],['Other'],russia clash erupt bashkortostan right activis...
2,EN_UA_300017.txt,Other,Other,"McDonald's to exit Russia, sell business in co...",EN,['Other'],['Other'],mcdonald exit russia sell business country ame...
3,EN_CC_100021.txt,Other,Other,"Collaborative plans, innovation keys to circul...",EN,['Other'],['Other'],collaborative plan innovation key circular rmg...
4,EN_UA_300041.txt,Other,Other,Russia intends to supply light ‘Mountain’ tank...,EN,['Other'],['Other'],russia intend supply light mountain tank infan...
...,...,...,...,...,...,...,...,...
870,PT_207.txt,CC: Criticism of institutions and authorities;...,CC: Criticism of institutions and authorities:...,Zequinha critica UE por adiar obrigatoriedade ...,PT,['CC: Criticism of institutions and authoritie...,['CC: Criticism of institutions and authoritie...,zequinha critico ue adiar obrigatoriedade pres...
871,PT_217.txt,CC: Amplifying Climate Fears;CC: Amplifying Cl...,CC: Amplifying Climate Fears: Other;CC: Amplif...,O que é a cúpula de calor? Entenda fenómeno qu...,PT,"['CC: Amplifying Climate Fears', 'CC: Amplifyi...","['CC: Amplifying Climate Fears: Other', 'CC: A...",cúpula calor entenda fenómeno prender ar quent...
872,PT_204.txt,CC: Amplifying Climate Fears;CC: Amplifying Cl...,CC: Amplifying Climate Fears: Amplifying exist...,COP28: Papa Francisco planeia participar nas n...,PT,"['CC: Amplifying Climate Fears', 'CC: Amplifyi...",['CC: Amplifying Climate Fears: Amplifying exi...,cop papa Francisco planear participar negociaç...
873,PT_229.txt,CC: Amplifying Climate Fears,CC: Amplifying Climate Fears: Amplifying exist...,Queda do tráfego pelo Canal do Panamá pode cus...,PT,['CC: Amplifying Climate Fears'],['CC: Amplifying Climate Fears: Amplifying exi...,queda tráfego canal panamá custar milhão dólar...


In [3]:

def tokenize(text):
    return text.lower().split()
# label mappings
def build_vocab_and_labels(data):
    word_to_idx = {"<PAD>": 0, "<UNK>": 1}
    label_to_idx = {"Other": 0}
    sublabel_to_idx = {"Other": 0}
    language_to_idx = {"EN": 0, "PT": 1}

    for _, row in data.iterrows():
        # index the vocab
        for word in tokenize(row["cleaned"]):
            if word not in word_to_idx:
                word_to_idx[word] = len(word_to_idx)
        # index the labels
        for label in row["narrative"].split(";"):
            if label not in label_to_idx:
                label_to_idx[label] = len(label_to_idx)
        for sublabel in row["sub_narrative"].split(";"):
            if sublabel not in sublabel_to_idx:
                sublabel_to_idx[sublabel] = len(sublabel_to_idx)

    return word_to_idx, label_to_idx, sublabel_to_idx, language_to_idx


In [4]:

word_to_idx, label_to_idx, sublabel_to_idx, language_to_idx = build_vocab_and_labels(data)
len(word_to_idx)

21812

In [5]:

len(label_to_idx)

22

In [6]:

len(sublabel_to_idx)


93

In [7]:

class DocumentDataset(Dataset):
    def __init__(self, df, word_to_idx, label_to_idx, sublabel_to_idx, language_to_idx, max_len=50):
        self.data = df
        self.word_to_idx = word_to_idx
        self.label_to_idx = label_to_idx
        self.sublabel_to_idx = sublabel_to_idx
        self.language_to_idx = language_to_idx
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        tokens = tokenize(row["text"])
        # assign the unk index if the word isn't indexed already
        input_ids = [self.word_to_idx.get(word, self.word_to_idx["<UNK>"]) for word in tokens]
        # make sure the length is always 50 through padding if needed
        input_ids = input_ids[:self.max_len] + [self.word_to_idx["<PAD>"]] * (self.max_len - len(input_ids))

        narrative_labels = [self.label_to_idx[label] for label in row["narrative"].split(";")]
        subnarrative_labels = [self.sublabel_to_idx[label] for label in row["sub_narrative"].split(";")]

        # Multi-label one-hot encoding
        narrative_targets = torch.zeros(len(self.label_to_idx))
        narrative_targets[narrative_labels] = 1

        subnarrative_targets = torch.zeros(len(self.sublabel_to_idx))
        subnarrative_targets[subnarrative_labels] = 1

        return (
            torch.tensor(input_ids, dtype=torch.long),
            torch.tensor(self.language_to_idx[row["language"]], dtype=torch.long),
            narrative_targets,
            subnarrative_targets
        )



In [8]:

train_dataset = DocumentDataset(training_data, word_to_idx, label_to_idx, sublabel_to_idx, language_to_idx)
val_dataset = DocumentDataset(target_data, word_to_idx, label_to_idx, sublabel_to_idx, language_to_idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [10]:

# LSTM Model with an embedding layer and three classifiers in th end
class LTSMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_languages, num_narratives, num_subnarratives):
        super(LTSMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.lang_classifier = nn.Linear(hidden_dim, num_languages)
        self.narrative_classifier = nn.Linear(hidden_dim, num_narratives)
        self.subnarrative_classifier = nn.Linear(hidden_dim, num_subnarratives)

    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.rnn(embedded)
        hidden = hidden.squeeze(0)

        lang_out = self.lang_classifier(hidden)
        narrative_out = self.narrative_classifier(hidden)
        subnarrative_out = self.subnarrative_classifier(hidden)

        return lang_out, narrative_out, subnarrative_out

In [11]:

vocab_size = len(word_to_idx)
embed_dim = 1024
hidden_dim = 1024
num_languages = len(language_to_idx)
num_narratives = len(label_to_idx)
num_subnarratives = len(sublabel_to_idx)

# the unweighted model to the gpu
model_unweighted = LTSMModel(vocab_size, embed_dim, hidden_dim, num_languages, num_narratives, num_subnarratives)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_unweighted.to(device)

# the unweighted loss fucntions
criterion_lang_unweighted = nn.CrossEntropyLoss()
criterion_bce_unweighted = nn.BCEWithLogitsLoss()
optimizer_unweighted = optim.Adam(model_unweighted.parameters(), lr=0.001)
# Calculate the number of parameters
total_params = sum(p.numel() for p in model_unweighted.parameters())
trainable_params = sum(p.numel() for p in model_unweighted.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")


Total parameters: 30852213
Trainable parameters: 30852213


In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Training Loop
num_epochs = 201
for epoch in range(num_epochs):
    model_unweighted.train()
    train_loss = 0
    all_lang_preds, all_lang_targets = [], []
    all_narrative_preds, all_narrative_targets = [], []
    all_subnarrative_preds, all_subnarrative_targets = [], []

    for inputs, lang_targets, narrative_targets, subnarrative_targets in train_loader:
        inputs, lang_targets, narrative_targets, subnarrative_targets = (
            inputs.to(device),
            lang_targets.to(device),
            narrative_targets.to(device),
            subnarrative_targets.to(device),
        )

        optimizer_unweighted.zero_grad()
        # forward pass
        lang_out, narrative_out, subnarrative_out = model_unweighted(inputs)

        # backward pass
        loss_lang = criterion_lang_unweighted(lang_out, lang_targets)
        loss_narrative = criterion_bce_unweighted(narrative_out, narrative_targets)
        loss_subnarrative = criterion_bce_unweighted(subnarrative_out, subnarrative_targets)

        loss = loss_lang + loss_narrative + loss_subnarrative
        loss.backward()
        optimizer_unweighted.step()
        train_loss += loss.item()

        # predictions
        lang_preds = lang_out.argmax(dim=1).cpu().numpy()
        all_lang_preds.extend(lang_preds)
        all_lang_targets.extend(lang_targets.cpu().numpy())

        narrative_preds = (torch.sigmoid(narrative_out) > 0.5).int().cpu().numpy()
        all_narrative_preds.extend(narrative_preds)
        all_narrative_targets.extend(narrative_targets.cpu().numpy())

        subnarrative_preds = (torch.sigmoid(subnarrative_out) > 0.5).int().cpu().numpy()
        all_subnarrative_preds.extend(subnarrative_preds)
        all_subnarrative_targets.extend(subnarrative_targets.cpu().numpy())

    # Metrics
    lang_accuracy = accuracy_score(all_lang_targets, all_lang_preds)
    lang_precision = precision_score(all_lang_targets, all_lang_preds, average="weighted")
    lang_recall = recall_score(all_lang_targets, all_lang_preds, average="weighted")
    lang_f1 = f1_score(all_lang_targets, all_lang_preds, average="weighted")
    lang_cm = confusion_matrix(all_lang_targets, all_lang_preds)

    narrative_macro_precision = precision_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)
    narrative_macro_recall = recall_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)
    narrative_macro_f1 = f1_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)

    narrative_micro_precision = precision_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)
    narrative_micro_recall = recall_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)
    narrative_micro_f1 = f1_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)

    subnarrative_macro_precision = precision_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)
    subnarrative_macro_recall = recall_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)
    subnarrative_macro_f1 = f1_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)

    subnarrative_micro_precision = precision_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)
    subnarrative_micro_recall = recall_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)
    subnarrative_micro_f1 = f1_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)

    # Log Epoch Results
    if epoch % 200 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss / len(train_loader):.4f}")
        print("\nLanguage Classification Metrics:")
        print(f"  Accuracy: {lang_accuracy:.4f}")
        print(f"  Precision: {lang_precision:.4f}")
        print(f"  Recall: {lang_recall:.4f}")
        print(f"  F1-Score: {lang_f1:.4f}")
        print(f"  Confusion Matrix:\n{lang_cm}")

        print("\nNarrative Classification Metrics (Macro Average):")
        print(f"  Precision: {narrative_macro_precision:.4f}")
        print(f"  Recall: {narrative_macro_recall:.4f}")
        print(f"  F1-Score: {narrative_macro_f1:.4f}")

        print("\nNarrative Classification Metrics (Micro Average):")
        print(f"  Precision: {narrative_micro_precision:.4f}")
        print(f"  Recall: {narrative_micro_recall:.4f}")
        print(f"  F1-Score: {narrative_micro_f1:.4f}")

        print("\nSub-Narrative Classification Metrics (Macro Average):")
        print(f"  Precision: {subnarrative_macro_precision:.4f}")
        print(f"  Recall: {subnarrative_macro_recall:.4f}")
        print(f"  F1-Score: {subnarrative_macro_f1:.4f}")

        print("\nSub-Narrative Classification Metrics (Micro Average):")
        print(f"  Precision: {subnarrative_micro_precision:.4f}")
        print(f"  Recall: {subnarrative_micro_recall:.4f}")
        print(f"  F1-Score: {subnarrative_micro_f1:.4f}")

        print("=" * 50)


Epoch 1/201
Train Loss: 0.8290

Language Classification Metrics:
  Accuracy: 0.8561
  Precision: 0.8566
  Recall: 0.8561
  F1-Score: 0.8560
  Confusion Matrix:
[[334  65]
 [ 50 350]]

Narrative Classification Metrics (Macro Average):
  Precision: 0.0697
  Recall: 0.0187
  F1-Score: 0.0262

Narrative Classification Metrics (Micro Average):
  Precision: 0.1551
  Recall: 0.0442
  F1-Score: 0.0688

Sub-Narrative Classification Metrics (Macro Average):
  Precision: 0.0228
  Recall: 0.0186
  F1-Score: 0.0173

Sub-Narrative Classification Metrics (Micro Average):
  Precision: 0.0476
  Recall: 0.0435
  F1-Score: 0.0455
Epoch 201/201
Train Loss: 0.0039

Language Classification Metrics:
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-Score: 1.0000
  Confusion Matrix:
[[399   0]
 [  0 400]]

Narrative Classification Metrics (Macro Average):
  Precision: 0.9832
  Recall: 0.9866
  F1-Score: 0.9849

Narrative Classification Metrics (Micro Average):
  Precision: 0.9866
  Recall: 0.9923
 

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Validation Loop
train_loss = 0
all_lang_preds, all_lang_targets = [], []
all_narrative_preds, all_narrative_targets = [], []
all_subnarrative_preds, all_subnarrative_targets = [], []

for inputs, lang_targets, narrative_targets, subnarrative_targets in val_loader:
    inputs, lang_targets, narrative_targets, subnarrative_targets = (
        inputs.to(device),
        lang_targets.to(device),
        narrative_targets.to(device),
        subnarrative_targets.to(device),
    )
    # forward pass
    lang_out, narrative_out, subnarrative_out = model_unweighted(inputs)

    # Losses
    loss_lang = criterion_lang_unweighted(lang_out, lang_targets)
    loss_narrative = criterion_bce_unweighted(narrative_out, narrative_targets)
    loss_subnarrative = criterion_bce_unweighted(subnarrative_out, subnarrative_targets)

    loss = loss_lang + loss_narrative + loss_subnarrative
    train_loss += loss.item()

    # predictions
    lang_preds = lang_out.argmax(dim=1).cpu().numpy()
    all_lang_preds.extend(lang_preds)
    all_lang_targets.extend(lang_targets.cpu().numpy())

    narrative_preds = (torch.sigmoid(narrative_out) > 0.5).int().cpu().numpy()
    all_narrative_preds.extend(narrative_preds)
    all_narrative_targets.extend(narrative_targets.cpu().numpy())

    subnarrative_preds = (torch.sigmoid(subnarrative_out) > 0.5).int().cpu().numpy()
    all_subnarrative_preds.extend(subnarrative_preds)
    all_subnarrative_targets.extend(subnarrative_targets.cpu().numpy())
    
#  Metrics
lang_accuracy = accuracy_score(all_lang_targets, all_lang_preds)
lang_precision = precision_score(all_lang_targets, all_lang_preds, average="weighted")
lang_recall = recall_score(all_lang_targets, all_lang_preds, average="weighted")
lang_f1 = f1_score(all_lang_targets, all_lang_preds, average="weighted")
lang_cm = confusion_matrix(all_lang_targets, all_lang_preds)

# Narrative Metrics
narrative_accuracy = accuracy_score(all_narrative_targets, all_narrative_preds)
narrative_macro_precision = precision_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)
narrative_macro_recall = recall_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)
narrative_macro_f1 = f1_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)

narrative_micro_accuracy = accuracy_score(all_narrative_targets, all_narrative_preds)
narrative_micro_precision = precision_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)
narrative_micro_recall = recall_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)
narrative_micro_f1 = f1_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)

# Sub-Narrative Metrics
subnarrative_accuracy = accuracy_score(all_subnarrative_targets, all_subnarrative_preds)
subnarrative_macro_precision = precision_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)
subnarrative_macro_recall = recall_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)
subnarrative_macro_f1 = f1_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)

subnarrative_micro_accuracy = accuracy_score(all_subnarrative_targets, all_subnarrative_preds)
subnarrative_micro_precision = precision_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)
subnarrative_micro_recall = recall_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)
subnarrative_micro_f1 = f1_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)

# Log Epoch Results
print(f"Validation Loss: {train_loss / len(val_loader):.4f}")
print("\nLanguage Classification Metrics:")
print(f"  Accuracy: {lang_accuracy:.4f}")
print(f"  Precision: {lang_precision:.4f}")
print(f"  Recall: {lang_recall:.4f}")
print(f"  F1-Score: {lang_f1:.4f}")
print(f"  Confusion Matrix:\n{lang_cm}")

print("\nNarrative Classification Metrics (Macro Average):")
print(f"  Accuracy: {narrative_accuracy:.4f}")
print(f"  Precision: {narrative_macro_precision:.4f}")
print(f"  Recall: {narrative_macro_recall:.4f}")
print(f"  F1-Score: {narrative_macro_f1:.4f}")

print("\nNarrative Classification Metrics (Micro Average):")
print(f"  Accuracy: {narrative_micro_accuracy:.4f}")
print(f"  Precision: {narrative_micro_precision:.4f}")
print(f"  Recall: {narrative_micro_recall:.4f}")
print(f"  F1-Score: {narrative_micro_f1:.4f}")

print("\nSub-Narrative Classification Metrics (Macro Average):")
print(f"  Accuracy: {subnarrative_accuracy:.4f}")
print(f"  Precision: {subnarrative_macro_precision:.4f}")
print(f"  Recall: {subnarrative_macro_recall:.4f}")
print(f"  F1-Score: {subnarrative_macro_f1:.4f}")

print("\nSub-Narrative Classification Metrics (Micro Average):")
print(f"  Accuracy: {subnarrative_micro_accuracy:.4f}")
print(f"  Precision: {subnarrative_micro_precision:.4f}")
print(f"  Recall: {subnarrative_micro_recall:.4f}")
print(f"  F1-Score: {subnarrative_micro_f1:.4f}")
print("=" * 50)



Validation Loss: 0.8558

Language Classification Metrics:
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-Score: 1.0000
  Confusion Matrix:
[[41  0]
 [ 0 35]]

Narrative Classification Metrics (Macro Average):
  Accuracy: 0.1711
  Precision: 0.2330
  Recall: 0.1061
  F1-Score: 0.1343

Narrative Classification Metrics (Micro Average):
  Accuracy: 0.1711
  Precision: 0.5088
  Recall: 0.1973
  F1-Score: 0.2843

Sub-Narrative Classification Metrics (Macro Average):
  Accuracy: 0.0789
  Precision: 0.0345
  Recall: 0.0217
  F1-Score: 0.0238

Sub-Narrative Classification Metrics (Micro Average):
  Accuracy: 0.0789
  Precision: 0.3023
  Recall: 0.0674
  F1-Score: 0.1102


In [14]:
import torch

all_dataset = DocumentDataset(data, word_to_idx, label_to_idx, sublabel_to_idx, language_to_idx)


# get the labels
narrative_targets = torch.stack([torch.tensor(target[2]) for target in all_dataset]).to(device)
subnarrative_targets = torch.stack([torch.tensor(target[3]) for target in all_dataset]).to(device)

# Compute the class weights for both narrative and sub_narrative
narrative_class_weights = narrative_targets.sum(dim=0)  # Sum along rows (class-wise sum)
subnarrative_class_weights = subnarrative_targets.sum(dim=0)  # Sum along rows (class-wise sum)

# Normalize the class weights by dividing by the maximum class weight
pos_weight_narrative = (narrative_class_weights.max() / narrative_class_weights).to(device)
pos_weight_subnarrative = (subnarrative_class_weights.max() / subnarrative_class_weights).to(device)


print("Narrative Class Weights:", narrative_class_weights)
print("Subnarrative Class Weights:", subnarrative_class_weights)
print("Pos Weight for Narrative:", pos_weight_narrative)
print("Pos Weight for Subnarrative:", pos_weight_subnarrative)

# update the loss functions
criterion_narrative_weighted = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight_narrative)
criterion_subnarrative_weighted = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight_subnarrative)



/tmp/ipykernel_21590/1170623561.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  narrative_targets = torch.stack([torch.tensor(target[2]) for target in all_dataset]).to(device)
/tmp/ipykernel_21590/1170623561.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  subnarrative_targets = torch.stack([torch.tensor(target[3]) for target in all_dataset]).to(device)


Narrative Class Weights: tensor([208.,  63.,  26.,  40., 129., 117.,  98.,  80., 171.,  14., 182.,  35.,
         20., 138.,  25.,  52.,  25.,  44.,   8.,  78., 148.,   7.],
       device='cuda:0')
Subnarrative Class Weights: tensor([208.,  23.,  16.,  29.,  30.,   6.,  24.,  27.,  19.,  51.,  24.,  78.,
         14.,  58.,  43.,  25.,  11., 127.,  38.,  35.,   1.,   2.,  13.,  10.,
         42.,  30.,  42.,  19.,   3.,  21.,  25.,  15.,  73.,  13.,   9.,  13.,
         11.,  11.,  27.,  18.,  51.,   2.,  76.,   6.,   5.,   5.,  14.,  31.,
         38.,  35.,  10.,  18.,  11.,   3.,   6.,  82.,  31.,  15.,  13.,  15.,
          8.,   3.,  32.,  25.,   3.,   4.,  24.,  20.,  18.,  28.,   1.,  38.,
          2.,  69.,   9.,   3.,   2.,   1.,   5.,   8.,   2.,   3.,  28., 106.,
         11.,   4.,  67.,   4.,   1.,   2.,   1.,   1.,   1.], device='cuda:0')
Pos Weight for Narrative: tensor([ 1.0000,  3.3016,  8.0000,  5.2000,  1.6124,  1.7778,  2.1224,  2.6000,
         1.2164, 14.8571,  1

In [15]:
# weighted model
model_weighted = LTSMModel(vocab_size, embed_dim, hidden_dim, num_languages, num_narratives, num_subnarratives)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_weighted.to(device)

criterion_lang_weighted = nn.CrossEntropyLoss()
optimizer_weighted = optim.Adam(model_weighted.parameters(), lr=0.001)

# Training Loop
num_epochs = 201
for epoch in range(num_epochs):
    model_weighted.train()
    train_loss = 0
    all_lang_preds, all_lang_targets = [], []
    all_narrative_preds, all_narrative_targets = [], []
    all_subnarrative_preds, all_subnarrative_targets = [], []

    for inputs, lang_targets, narrative_targets, subnarrative_targets in train_loader:
        inputs, lang_targets, narrative_targets, subnarrative_targets = (
            inputs.to(device),
            lang_targets.to(device),
            narrative_targets.to(device),
            subnarrative_targets.to(device),
        )

        optimizer_weighted.zero_grad()
        # forward pass
        lang_out, narrative_out, subnarrative_out = model_weighted(inputs)

        # backward pass
        loss_lang = criterion_lang_weighted(lang_out, lang_targets)
        loss_narrative = criterion_narrative_weighted(narrative_out, narrative_targets)
        loss_subnarrative = criterion_subnarrative_weighted(subnarrative_out, subnarrative_targets)

        loss = loss_lang + loss_narrative + loss_subnarrative
        loss.backward()
        optimizer_weighted.step()
        train_loss += loss.item()

        # predictions
        lang_preds = lang_out.argmax(dim=1).cpu().numpy()
        all_lang_preds.extend(lang_preds)
        all_lang_targets.extend(lang_targets.cpu().numpy())

        narrative_preds = (torch.sigmoid(narrative_out) > 0.5).int().cpu().numpy()
        all_narrative_preds.extend(narrative_preds)
        all_narrative_targets.extend(narrative_targets.cpu().numpy())

        subnarrative_preds = (torch.sigmoid(subnarrative_out) > 0.5).int().cpu().numpy()
        all_subnarrative_preds.extend(subnarrative_preds)
        all_subnarrative_targets.extend(subnarrative_targets.cpu().numpy())

    #  Metrics
    lang_accuracy = accuracy_score(all_lang_targets, all_lang_preds)
    lang_precision = precision_score(all_lang_targets, all_lang_preds, average="weighted")
    lang_recall = recall_score(all_lang_targets, all_lang_preds, average="weighted")
    lang_f1 = f1_score(all_lang_targets, all_lang_preds, average="weighted")
    lang_cm = confusion_matrix(all_lang_targets, all_lang_preds)

    # Compute Metrics for Narrative (Multi-Label) Classification
    narrative_precision = precision_score(all_narrative_targets, all_narrative_preds, average="weighted", zero_division=0)
    narrative_recall = recall_score(all_narrative_targets, all_narrative_preds, average="weighted", zero_division=0)
    narrative_f1 = f1_score(all_narrative_targets, all_narrative_preds, average="weighted", zero_division=0)

    # Compute Metrics for Sub-Narrative (Multi-Label) Classification
    subnarrative_precision = precision_score(all_subnarrative_targets, all_subnarrative_preds, average="weighted", zero_division=0)
    subnarrative_recall = recall_score(all_subnarrative_targets, all_subnarrative_preds, average="weighted", zero_division=0)
    subnarrative_f1 = f1_score(all_subnarrative_targets, all_subnarrative_preds, average="weighted", zero_division=0)

    # Log Epoch Results
    if epoch % 200 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss / len(train_loader):.4f}")
        print("\nLanguage Classification Metrics:")
        print(f"  Accuracy: {lang_accuracy:.4f}")
        print(f"  Precision: {lang_precision:.4f}")
        print(f"  Recall: {lang_recall:.4f}")
        print(f"  F1-Score: {lang_f1:.4f}")
        print(f"  Confusion Matrix:\n{lang_cm}")

        print("\nNarrative Classification Metrics (Macro Average):")
        print(f"  Precision: {narrative_macro_precision:.4f}")
        print(f"  Recall: {narrative_macro_recall:.4f}")
        print(f"  F1-Score: {narrative_macro_f1:.4f}")

        print("\nNarrative Classification Metrics (Micro Average):")
        print(f"  Precision: {narrative_micro_precision:.4f}")
        print(f"  Recall: {narrative_micro_recall:.4f}")
        print(f"  F1-Score: {narrative_micro_f1:.4f}")

        print("\nSub-Narrative Classification Metrics (Macro Average):")
        print(f"  Precision: {subnarrative_macro_precision:.4f}")
        print(f"  Recall: {subnarrative_macro_recall:.4f}")
        print(f"  F1-Score: {subnarrative_macro_f1:.4f}")

        print("\nSub-Narrative Classification Metrics (Micro Average):")
        print(f"  Precision: {subnarrative_micro_precision:.4f}")
        print(f"  Recall: {subnarrative_micro_recall:.4f}")
        print(f"  F1-Score: {subnarrative_micro_f1:.4f}")

        print("=" * 50)


Epoch 1/201
Train Loss: 1.5537

Language Classification Metrics:
  Accuracy: 0.8598
  Precision: 0.8638
  Recall: 0.8598
  F1-Score: 0.8594
  Confusion Matrix:
[[322  77]
 [ 35 365]]

Narrative Classification Metrics (Macro Average):
  Precision: 0.2330
  Recall: 0.1061
  F1-Score: 0.1343

Narrative Classification Metrics (Micro Average):
  Precision: 0.5088
  Recall: 0.1973
  F1-Score: 0.2843

Sub-Narrative Classification Metrics (Macro Average):
  Precision: 0.0345
  Recall: 0.0217
  F1-Score: 0.0238

Sub-Narrative Classification Metrics (Micro Average):
  Precision: 0.3023
  Recall: 0.0674
  F1-Score: 0.1102
Epoch 201/201
Train Loss: 0.0055

Language Classification Metrics:
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-Score: 1.0000
  Confusion Matrix:
[[399   0]
 [  0 400]]

Narrative Classification Metrics (Macro Average):
  Precision: 0.2330
  Recall: 0.1061
  F1-Score: 0.1343

Narrative Classification Metrics (Micro Average):
  Precision: 0.5088
  Recall: 0.1973
 

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Validation Loop
train_loss = 0
all_lang_preds, all_lang_targets = [], []
all_narrative_preds, all_narrative_targets = [], []
all_subnarrative_preds, all_subnarrative_targets = [], []

for inputs, lang_targets, narrative_targets, subnarrative_targets in val_loader:
    inputs, lang_targets, narrative_targets, subnarrative_targets = (
        inputs.to(device),
        lang_targets.to(device),
        narrative_targets.to(device),
        subnarrative_targets.to(device),
    )
    # forward pass
    lang_out, narrative_out, subnarrative_out = model_weighted(inputs)

    # backward pass
    loss_lang = criterion_lang_weighted(lang_out, lang_targets)
    loss_narrative = criterion_narrative_weighted(narrative_out, narrative_targets)
    loss_subnarrative = criterion_subnarrative_weighted(subnarrative_out, subnarrative_targets)

    loss = loss_lang + loss_narrative + loss_subnarrative
    train_loss += loss.item()

    # predictions
    lang_preds = lang_out.argmax(dim=1).cpu().numpy()
    all_lang_preds.extend(lang_preds)
    all_lang_targets.extend(lang_targets.cpu().numpy())

    narrative_preds = (torch.sigmoid(narrative_out) > 0.5).int().cpu().numpy()
    all_narrative_preds.extend(narrative_preds)
    all_narrative_targets.extend(narrative_targets.cpu().numpy())

    subnarrative_preds = (torch.sigmoid(subnarrative_out) > 0.5).int().cpu().numpy()
    all_subnarrative_preds.extend(subnarrative_preds)
    all_subnarrative_targets.extend(subnarrative_targets.cpu().numpy())

#  Metrics
lang_accuracy = accuracy_score(all_lang_targets, all_lang_preds)
lang_precision = precision_score(all_lang_targets, all_lang_preds, average="weighted")
lang_recall = recall_score(all_lang_targets, all_lang_preds, average="weighted")
lang_f1 = f1_score(all_lang_targets, all_lang_preds, average="weighted")
lang_cm = confusion_matrix(all_lang_targets, all_lang_preds)

# Narrative Metrics
narrative_accuracy = accuracy_score(all_narrative_targets, all_narrative_preds)
narrative_macro_precision = precision_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)
narrative_macro_recall = recall_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)
narrative_macro_f1 = f1_score(all_narrative_targets, all_narrative_preds, average="macro", zero_division=0)

narrative_micro_accuracy = accuracy_score(all_narrative_targets, all_narrative_preds)
narrative_micro_precision = precision_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)
narrative_micro_recall = recall_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)
narrative_micro_f1 = f1_score(all_narrative_targets, all_narrative_preds, average="micro", zero_division=0)

# Sub-Narrative Metrics
subnarrative_accuracy = accuracy_score(all_subnarrative_targets, all_subnarrative_preds)
subnarrative_macro_precision = precision_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)
subnarrative_macro_recall = recall_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)
subnarrative_macro_f1 = f1_score(all_subnarrative_targets, all_subnarrative_preds, average="macro", zero_division=0)

subnarrative_micro_accuracy = accuracy_score(all_subnarrative_targets, all_subnarrative_preds)
subnarrative_micro_precision = precision_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)
subnarrative_micro_recall = recall_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)
subnarrative_micro_f1 = f1_score(all_subnarrative_targets, all_subnarrative_preds, average="micro", zero_division=0)

# Log Epoch Results
print(f"Validation Loss: {train_loss / len(val_loader):.4f}")
print("\nLanguage Classification Metrics:")
print(f"  Accuracy: {lang_accuracy:.4f}")
print(f"  Precision: {lang_precision:.4f}")
print(f"  Recall: {lang_recall:.4f}")
print(f"  F1-Score: {lang_f1:.4f}")
print(f"  Confusion Matrix:\n{lang_cm}")

print("\nNarrative Classification Metrics (Macro Average):")
print(f"  Accuracy: {narrative_accuracy:.4f}")
print(f"  Precision: {narrative_macro_precision:.4f}")
print(f"  Recall: {narrative_macro_recall:.4f}")
print(f"  F1-Score: {narrative_macro_f1:.4f}")

print("\nNarrative Classification Metrics (Micro Average):")
print(f"  Accuracy: {narrative_micro_accuracy:.4f}")
print(f"  Precision: {narrative_micro_precision:.4f}")
print(f"  Recall: {narrative_micro_recall:.4f}")
print(f"  F1-Score: {narrative_micro_f1:.4f}")

print("\nSub-Narrative Classification Metrics (Macro Average):")
print(f"  Accuracy: {subnarrative_accuracy:.4f}")
print(f"  Precision: {subnarrative_macro_precision:.4f}")
print(f"  Recall: {subnarrative_macro_recall:.4f}")
print(f"  F1-Score: {subnarrative_macro_f1:.4f}")

print("\nSub-Narrative Classification Metrics (Micro Average):")
print(f"  Accuracy: {subnarrative_micro_accuracy:.4f}")
print(f"  Precision: {subnarrative_micro_precision:.4f}")
print(f"  Recall: {subnarrative_micro_recall:.4f}")
print(f"  F1-Score: {subnarrative_micro_f1:.4f}")
print("=" * 50)



Validation Loss: 4.8377

Language Classification Metrics:
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-Score: 1.0000
  Confusion Matrix:
[[41  0]
 [ 0 35]]

Narrative Classification Metrics (Macro Average):
  Accuracy: 0.1974
  Precision: 0.2928
  Recall: 0.1488
  F1-Score: 0.1815

Narrative Classification Metrics (Micro Average):
  Accuracy: 0.1974
  Precision: 0.5373
  Recall: 0.2449
  F1-Score: 0.3364

Sub-Narrative Classification Metrics (Macro Average):
  Accuracy: 0.1053
  Precision: 0.0625
  Recall: 0.0385
  F1-Score: 0.0415

Sub-Narrative Classification Metrics (Micro Average):
  Accuracy: 0.1053
  Precision: 0.3774
  Recall: 0.1036
  F1-Score: 0.1626
